# deepSimDEF for Prediction of Protein-Protein Interactions (PPI)

## Importing the required libraries

In [1]:
import os
import sys
import random
import operator
import numpy as np
import keras.backend as K

from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics import f1_score

from data_provider_PPI import gene_pair_data_reader, input_data_maker
from deepSimDEF_network import deepSimDEF_PPI
from deepSimDEF_save import save_simDEF_model, save_simDEF_embeddings

np.random.seed(123)


Using TensorFlow backend.


## Setting variables, reading GO annotations of genes, and preparing them for networks

In [2]:
FOLD = 10
DROPOUT = 0.3
MAX_POOL = True

PRE_TRAINED = True
UPDATABLE = True

ACTIVATION_HIDDEN = 'relu'
ACTIVATION_HIGHWAY = 'sigmoid'
ACTIVATION_OUTPUT = 'sigmoid'

EMBEDDING_DIM = 100
NB_EPOCH = 250
BATCH_SIZE = 256
OPTIMIZER = 'adadelta'

IEA = True
SEQ = False

TRANSFER_LEARNING = False

SAVE_MODEL = True
SAVE_EMBEDDINGS = True

SUB_ONTOLOGY = ['BP', 'CC', 'MF']
SUB_ONTOLOGY_work = ['BP', 'CC', 'MF']

WITH_HIGH_THROUPUT = False

SBOs = {}
for sbo in SUB_ONTOLOGY_work:
    if sbo == 'BP':
        SBOs[sbo] = 'Biolobical Process (BP)'
    elif sbo == 'CC':
        SBOs[sbo] = 'Cellular Component (CC)'
    elif sbo == 'MF':
        SBOs[sbo] = 'Molecular Function (MF)'
    
WE = {}
embedding_save = {}
MAX_SEQUENCE_LENGTH = {}
MAX_SEQUENCE_LENGTH_INDEX = {}
sequences = {}
word_indeces = {}
protein_index = {}    
    
for sbo in SUB_ONTOLOGY:
    WE[sbo] = 'GO_' + sbo + '_Embeddings_100D_0_6.emb'
    embedding_save[sbo] = 'GO_' + sbo + '_Embeddings_100D_0_6_updated.emb'
    MAX_SEQUENCE_LENGTH[sbo] = 0
    MAX_SEQUENCE_LENGTH_INDEX[sbo] = []
    sequences[sbo] = []
    word_indeces[sbo] = []
    protein_index[sbo] = {}
    
    if IEA:
        file_reader = open('gene_product_GO_terms_with_IEA' + '.' + sbo)
    else:
        file_reader = open('gene_product_GO_terms_without_IEA' + '.' + sbo)
    
    index_counter = 1
    texts = []
    for line in file_reader:
        values = line.rstrip().replace(':', '').split()
        protein_index[sbo][values[0]] = index_counter
        if len(values[1:]) > MAX_SEQUENCE_LENGTH[sbo]:
            MAX_SEQUENCE_LENGTH[sbo] = len(values[1:])
            MAX_SEQUENCE_LENGTH_INDEX[sbo] = index_counter
        texts.append(' '.join(values[1:]))
        index_counter += 1
        
    tokenizer = Tokenizer(lower=False, num_words=0)
    tokenizer.fit_on_texts(texts)
    sequences[sbo] = tokenizer.texts_to_sequences(texts)

    word_indeces[sbo] = tokenizer.word_index
    
    if sbo == 'BP':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Biolobical Process (BP) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    elif sbo == 'CC':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Cellular Component (CC) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    elif sbo == 'MF':
        print "\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Molecular Function (MF) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
            
    print "Found " + str(len(word_indeces[sbo])) + " unique tokens in " + sbo

    MOST_FREQUENT_LEVEL = 10
    print 'Top', MOST_FREQUENT_LEVEL, 'Most Frequent GO terms annotating sequences in', sbo + ":"
    for GO_ID, indx in sorted(word_indeces[sbo].items(), key=operator.itemgetter(1))[:MOST_FREQUENT_LEVEL]:
        print '  >>>', GO_ID, '   ' ,indx
        
    print "Number of annotated gene products by '" + sbo + "' terms: " + str(len(sequences[sbo]))
    print "Maximum annotation length of one gene product ('" + sbo + "' sub-ontology):", MAX_SEQUENCE_LENGTH[sbo]
    print "Index/line of the gene product with maximum annotations ('" + sbo + "' sub-ontology):", MAX_SEQUENCE_LENGTH_INDEX[sbo]
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n"
    
    file_reader.close()
    
    
fully_annotated_sequences = []   # we keep only those genes for which we have annatoation from all ontologies (defined in SUB_ONTOLOGY variable)
for sbo in SUB_ONTOLOGY:
    fully_annotated_sequences.append(protein_index[sbo].keys())
fully_annotated_sequences = list(set(fully_annotated_sequences[0]).intersection(*fully_annotated_sequences))
print "Number of fully annotated gene products:", len(fully_annotated_sequences)



~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Biolobical Process (BP) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 3054 unique tokens in BP
Top 10 Most Frequent GO terms annotating sequences in BP:
  >>> GO0006810     1
  >>> GO0006351     2
  >>> GO0006355     3
  >>> GO0015031     4
  >>> GO0055114     5
  >>> GO0007049     6
  >>> GO0006414     7
  >>> GO0008152     8
  >>> GO0006412     9
  >>> GO0055085     10
Number of annotated gene products by 'BP' terms: 5680
Maximum annotation length of one gene product ('BP' sub-ontology): 44
Index/line of the gene product with maximum annotations ('BP' sub-ontology): 294
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Cellular Component (CC) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Found 782 unique tokens in CC
Top 10 Most Frequent GO terms annotating sequences in CC:
  >>> GO0005737     1
  >>> GO0005634     2
  >>> GO0016020     3
  >>> GO0016021     4
  >>> GO0005739     5
  >>> GO0005829     6
  >>> GO0005783 

## Reading the gene-pair PPIs: manually curated PPIs and high-throughput PPIs (optional)

In [3]:

input_data_dir = '3.PPI_data/IEA+/PPI_FULL_physical_interactions_manually_curated'
annotation_G1_dic_MC, annotation_G2_dic_MC, interaction_pr_list_MC = gene_pair_data_reader(data_dir=input_data_dir, 
                                                                                           SUB_ONTOLOGY_work=SUB_ONTOLOGY_work, 
                                                                                           fully_annotated_sequences=fully_annotated_sequences, 
                                                                                           sequences=sequences, 
                                                                                           protein_index=protein_index,
                                                                                           MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH)


VALIDATION_SPLIT = 1.0/FOLD
indices = np.arange(annotation_G1_dic_MC[sbo].shape[0])
np.random.shuffle(indices)
test_size = int(VALIDATION_SPLIT * annotation_G1_dic_MC[sbo].shape[0])


annotation_G1_dic_HT = []
annotation_G2_dic_HT = []
interaction_pr_list_HT = []

if WITH_HIGH_THROUPUT:
    input_data_dir = '3.PPI_data/IEA+/PPI_FULL_physical_interactions_high_throughput'
    annotation_G1_dic_HT, annotation_G2_dic_HT, interaction_pr_list_HT = gene_pair_data_reader(data_dir=input_data_dir, 
                                                                                               SUB_ONTOLOGY_work=SUB_ONTOLOGY_work, 
                                                                                               fully_annotated_sequences=fully_annotated_sequences, 
                                                                                               sequences=sequences, 
                                                                                               protein_index=protein_index,
                                                                                               MAX_SEQUENCE_LENGTH= MAX_SEQUENCE_LENGTH)


Shape of data tensor 1 (BP): (32956, 44)
Shape of data tensor 2 (BP): (32956, 44)
Shape of similarity tensor (BP): (32956,) 

Shape of data tensor 1 (CC): (32956, 17)
Shape of data tensor 2 (CC): (32956, 17)
Shape of similarity tensor (CC): (32956,) 

Shape of data tensor 1 (MF): (32956, 33)
Shape of data tensor 2 (MF): (32956, 33)
Shape of similarity tensor (MF): (32956,) 

Number of positive classes/interactions: 16478


## printing some information about the setting of the network and the experiment

In [4]:
for sbo in SUB_ONTOLOGY_work:
    print "@@@ " + SBOs[sbo] + " @@@"

if IEA:
    print "^^^ With IEA ^^^"
else:
    print "^^^ Without IEA ^^^"

print "%%% Optimizer:", OPTIMIZER, "%%%"

if PRE_TRAINED:
    if UPDATABLE:
        print "+++ Pre-trained (updatable) +++"
    else:
        print "+++ Pre-trained (not updatable) +++"
else:
    print "+++ NOT Pre-trained +++"

@@@ Biolobical Process (BP) @@@
@@@ Cellular Component (CC) @@@
@@@ Molecular Function (MF) @@@
^^^ With IEA ^^^
%%% Optimizer: adadelta %%%
+++ Pre-trained (updatable) +++


## Making a 10-fold cross-validation experiment

In [5]:
models = []
embedding_layers = []
bests = []
thresholds = []
B = []

for m in range(0, FOLD):
    network = deepSimDEF_PPI(EMBEDDING_DIM, 
                             model_ind=m, 
                             MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH, 
                             WORD_EMBEDDINGS=WE,
                             SUB_ONTOLOGY_work=SUB_ONTOLOGY_work,
                             word_indeces=word_indeces, 
                             ACTIVATION_HIDDEN=ACTIVATION_HIDDEN, 
                             ACTIVATION_HIGHWAY=ACTIVATION_HIGHWAY, 
                             ACTIVATION_OUTPUT=ACTIVATION_OUTPUT, 
                             DROPOUT=DROPOUT, 
                             OPTIMIZER=OPTIMIZER)
    models.append(network[0])
    embedding_layers.append(network[1])
    bests.append(0)
    thresholds.append(0)
    B.append({})

Loaded 29375 word vectors for BP (Model 1)
Loaded 4046 word vectors for CC (Model 1)
Loaded 10541 word vectors for MF (Model 1)


/root/anaconda2/lib/python2.7/site-packages/keras/legacy/layers.py:652: UserWarning: The `Highway` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `Highway` layer is deprecated '


Model for Fold Number 1 Instantiated!!

Loaded 29375 word vectors for BP (Model 2)
Loaded 4046 word vectors for CC (Model 2)
Loaded 10541 word vectors for MF (Model 2)
Model for Fold Number 2 Instantiated!!

Loaded 29375 word vectors for BP (Model 3)
Loaded 4046 word vectors for CC (Model 3)
Loaded 10541 word vectors for MF (Model 3)
Model for Fold Number 3 Instantiated!!

Loaded 29375 word vectors for BP (Model 4)
Loaded 4046 word vectors for CC (Model 4)
Loaded 10541 word vectors for MF (Model 4)
Model for Fold Number 4 Instantiated!!

Loaded 29375 word vectors for BP (Model 5)
Loaded 4046 word vectors for CC (Model 5)
Loaded 10541 word vectors for MF (Model 5)
Model for Fold Number 5 Instantiated!!

Loaded 29375 word vectors for BP (Model 6)
Loaded 4046 word vectors for CC (Model 6)
Loaded 10541 word vectors for MF (Model 6)
Model for Fold Number 6 Instantiated!!

Loaded 29375 word vectors for BP (Model 7)
Loaded 4046 word vectors for CC (Model 7)
Loaded 10541 word vectors for MF (M

## Training the deepSimDEF netwrok for PPI task

In [6]:
RES = {}
best_total_f1 = 0
best_threshold = 0

early_stopping = EarlyStopping(monitor='val_loss', patience = 3)
cor = {}

best_epoch = 0

def pred(A, treshold = 0.5):
    B = []
    for n in A:
        if treshold < n:
            B.append(1)
        else:
            B.append(0)
    return B

def run_my_model(model_index, seq):
    X_train, y_train, X_test, y_test = input_data_maker(model_id=model_index, 
                                                        test_size=test_size, 
                                                        indices=indices, 
                                                        annotation_G1_dic_MC=annotation_G1_dic_MC, 
                                                        annotation_G2_dic_MC=annotation_G2_dic_MC, 
                                                        interaction_pr_list_MC=interaction_pr_list_MC, 
                                                        annotation_G1_dic_HT=annotation_G1_dic_HT,
                                                        annotation_G2_dic_HT=annotation_G2_dic_HT,
                                                        interaction_pr_list_HT=interaction_pr_list_HT,
                                                        SUB_ONTOLOGY_work=SUB_ONTOLOGY_work,
                                                        WITH_HIGH_THROUPUT=False)
    model = models[model_index]
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=1, validation_data=(X_test,y_test))
    p =  model.predict(X_test)
    for i in seq:
        predictions = np.asarray(pred(p, i))
        B[model_index][i] = np.round(f1_score(y_test, predictions, average='binary'), 5)
    pr = max(B[model_index].iteritems(), key=operator.itemgetter(1))[1]
    thresholds[model_index] = max(B[model_index].iteritems(), key=operator.itemgetter(1))[0]
    st = ''
    b = bests[model_index]
    if bests[model_index] < pr: 
        bests[model_index] = pr
        treshold = thresholds[model_index]
        st = "+ " + str(bests[model_index])
    else:
        st = "- " + str(bests[model_index])
    print ">>> F1-score (" + str(model_index + 1) + "):", pr, "Best (" + str(model_index + 1) + "):", st, "(" + str(thresholds[model_index]) + " : " + str(np.round(pr - b, 5)) + ")" + "\n"

def get_results(epoch_no):
    for i in seq:
        RES[i] = 0
        for j in range(FOLD):
            RES[i] += B[j][i]/FOLD
    res = max(RES.iteritems(), key=operator.itemgetter(1))[1]
    threshold_res = max(RES.iteritems(), key=operator.itemgetter(1))[0]
    cor[epoch_no + 1] = res
    total_max = 0
    for i, j in sorted(cor.items(), key=operator.itemgetter(1)):
        if total_max < j:
            total_max = j
            best_epoch = i
            threshold_best = threshold_res
    
    print "F1-score for this epoch:", res, "(", threshold_res, ")-- Best F1-score::==>", str(total_max), "(", threshold_best, ")  (for epoch #", str(best_epoch), "of", str(NB_EPOCH), "epochs)" + "\n"

def get_final_result():
    final_max = 0
    best_epoch = 0
    for i, j in sorted(cor.items(), key=operator.itemgetter(1)):
        if final_max < j:
            final_max = j
            best_epoch = i
        
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~ FINAL RESULT ~~~~~~~~~~~~~~~~~~~~~~~~~~~~" + "\n" 
    print "For embedding size '" + str(EMBEDDING_DIM) + "' best number of epochs is '" + str(i) + "' with the pearson correlation of: " + str(final_max) +"\n"
    
for e in range(NB_EPOCH):
     
    print "~~~~~~~~~ " + '/'.join(SUB_ONTOLOGY_work) +" ~~~~~~~~~~~~~~ EPOCH " + str(e + 1) + "/" + str(NB_EPOCH) + " (Embedding dimention: " + str(EMBEDDING_DIM) + ") ~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n" 
    seq = [0.5]
    if SEQ:
        seq = np.arange(0.11, 0.9, 0.01)
        
    for index in range(0, len(models)):
        run_my_model(index, seq)
    
    get_results(e)

get_final_result()

~~~~~~~~~ BP/CC/MF ~~~~~~~~~~~~~~ EPOCH 1/250 (Embedding dimention: 100) ~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5526 - fmeasure: 0.7296 - val_loss: 0.4168 - val_fmeasure: 0.8348
>>> F1-score (1): 0.83508 Best (1): + 0.83508 (0.5 : 0.83508)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5492 - fmeasure: 0.7321 - val_loss: 0.4406 - val_fmeasure: 0.7919
>>> F1-score (2): 0.79373 Best (2): + 0.79373 (0.5 : 0.79373)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5553 - fmeasure: 0.7264 - val_loss: 0.4252 - val_fmeasure: 0.8147
>>> F1-score (3): 0.81484 Best (3): + 0.81484 (0.5 : 0.81484)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.5526 - fmeasure: 0.7293 - val_loss

59322/59322 [==============================] - 3s - loss: 0.3643 - fmeasure: 0.8442 - val_loss: 0.3240 - val_fmeasure: 0.8725
>>> F1-score (1): 0.87315 Best (1): + 0.87315 (0.5 : 0.02)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.3612 - fmeasure: 0.8460 - val_loss: 0.3379 - val_fmeasure: 0.8503
>>> F1-score (2): 0.85106 Best (2): + 0.85106 (0.5 : 0.00426)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.3641 - fmeasure: 0.8456 - val_loss: 0.3262 - val_fmeasure: 0.8620
>>> F1-score (3): 0.86183 Best (3): + 0.86183 (0.5 : 0.00697)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.3600 - fmeasure: 0.8473 - val_loss: 0.3228 - val_fmeasure: 0.8699
>>> F1-score (4): 0.87035 Best (4): + 0.87035 (0.5 : 0.00768)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322

59322/59322 [==============================] - 3s - loss: 0.3043 - fmeasure: 0.8754 - val_loss: 0.2879 - val_fmeasure: 0.8837
>>> F1-score (1): 0.88438 Best (1): - 0.88478 (0.5 : -0.0004)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.3013 - fmeasure: 0.8756 - val_loss: 0.2864 - val_fmeasure: 0.8857
>>> F1-score (2): 0.88617 Best (2): + 0.88617 (0.5 : 0.00973)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.3028 - fmeasure: 0.8756 - val_loss: 0.2796 - val_fmeasure: 0.8877
>>> F1-score (3): 0.88737 Best (3): + 0.88737 (0.5 : 0.00768)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 4s - loss: 0.2988 - fmeasure: 0.8782 - val_loss: 0.2878 - val_fmeasure: 0.8876
>>> F1-score (4): 0.88759 Best (4): + 0.88759 (0.5 : 0.00878)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59

59322/59322 [==============================] - 3s - loss: 0.2603 - fmeasure: 0.8952 - val_loss: 0.2616 - val_fmeasure: 0.8988
>>> F1-score (1): 0.89937 Best (1): + 0.89937 (0.5 : 0.00543)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2598 - fmeasure: 0.8953 - val_loss: 0.2669 - val_fmeasure: 0.8924
>>> F1-score (2): 0.89305 Best (2): - 0.89705 (0.5 : -0.004)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2633 - fmeasure: 0.8936 - val_loss: 0.2549 - val_fmeasure: 0.9029
>>> F1-score (3): 0.90279 Best (3): + 0.90279 (0.5 : 0.00253)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2573 - fmeasure: 0.8961 - val_loss: 0.2666 - val_fmeasure: 0.8951
>>> F1-score (4): 0.89532 Best (4): + 0.89532 (0.5 : 0.0029)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/5932

59322/59322 [==============================] - 3s - loss: 0.2308 - fmeasure: 0.9078 - val_loss: 0.2668 - val_fmeasure: 0.8979
>>> F1-score (1): 0.89841 Best (1): - 0.90282 (0.5 : -0.00441)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2325 - fmeasure: 0.9071 - val_loss: 0.2505 - val_fmeasure: 0.9075
>>> F1-score (2): 0.90805 Best (2): + 0.90805 (0.5 : 0.00349)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2337 - fmeasure: 0.9095 - val_loss: 0.2433 - val_fmeasure: 0.9050
>>> F1-score (3): 0.90502 Best (3): + 0.90502 (0.5 : 0.00178)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.2309 - fmeasure: 0.9099 - val_loss: 0.2692 - val_fmeasure: 0.8997
>>> F1-score (4): 0.90006 Best (4): - 0.90006 (0.5 : 0.0)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322

...

59322/59322 [==============================] - 3s - loss: 0.0179 - fmeasure: 0.9936 - val_loss: 0.5048 - val_fmeasure: 0.9202
>>> F1-score (1): 0.92119 Best (1): - 0.924 (0.5 : -0.00281)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0179 - fmeasure: 0.9938 - val_loss: 0.5078 - val_fmeasure: 0.9207
>>> F1-score (2): 0.92114 Best (2): - 0.92359 (0.5 : -0.00245)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0173 - fmeasure: 0.9937 - val_loss: 0.5199 - val_fmeasure: 0.9213
>>> F1-score (3): 0.92156 Best (3): - 0.92595 (0.5 : -0.00439)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0188 - fmeasure: 0.9936 - val_loss: 0.5445 - val_fmeasure: 0.9121
>>> F1-score (4): 0.91234 Best (4): - 0.91731 (0.5 : -0.00497)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/

59322/59322 [==============================] - 3s - loss: 0.0184 - fmeasure: 0.9936 - val_loss: 0.4908 - val_fmeasure: 0.9200
>>> F1-score (1): 0.9208 Best (1): - 0.924 (0.5 : -0.0032)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0183 - fmeasure: 0.9930 - val_loss: 0.5028 - val_fmeasure: 0.9204
>>> F1-score (2): 0.92087 Best (2): - 0.92359 (0.5 : -0.00272)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0177 - fmeasure: 0.9935 - val_loss: 0.5055 - val_fmeasure: 0.9240
>>> F1-score (3): 0.92429 Best (3): - 0.92595 (0.5 : -0.00166)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0171 - fmeasure: 0.9941 - val_loss: 0.5618 - val_fmeasure: 0.9154
>>> F1-score (4): 0.91571 Best (4): - 0.91731 (0.5 : -0.0016)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/593

59322/59322 [==============================] - 3s - loss: 0.0160 - fmeasure: 0.9940 - val_loss: 0.5126 - val_fmeasure: 0.9207
>>> F1-score (1): 0.92161 Best (1): - 0.92464 (0.5 : -0.00303)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0175 - fmeasure: 0.9935 - val_loss: 0.5043 - val_fmeasure: 0.9185
>>> F1-score (2): 0.91894 Best (2): - 0.92359 (0.5 : -0.00465)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0178 - fmeasure: 0.9940 - val_loss: 0.5174 - val_fmeasure: 0.9252
>>> F1-score (3): 0.92544 Best (3): - 0.92595 (0.5 : -0.00051)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0173 - fmeasure: 0.9940 - val_loss: 0.5443 - val_fmeasure: 0.9132
>>> F1-score (4): 0.91344 Best (4): - 0.91731 (0.5 : -0.00387)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

59322/59322 [==============================] - 3s - loss: 0.0176 - fmeasure: 0.9938 - val_loss: 0.5204 - val_fmeasure: 0.9192
>>> F1-score (1): 0.9201 Best (1): - 0.92464 (0.5 : -0.00454)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0163 - fmeasure: 0.9941 - val_loss: 0.5048 - val_fmeasure: 0.9223
>>> F1-score (2): 0.92289 Best (2): - 0.92359 (0.5 : -0.0007)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0174 - fmeasure: 0.9939 - val_loss: 0.5137 - val_fmeasure: 0.9230
>>> F1-score (3): 0.92321 Best (3): - 0.92595 (0.5 : -0.00274)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0176 - fmeasure: 0.9939 - val_loss: 0.5615 - val_fmeasure: 0.9132
>>> F1-score (4): 0.91334 Best (4): - 0.91731 (0.5 : -0.00397)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/

59322/59322 [==============================] - 3s - loss: 0.0175 - fmeasure: 0.9936 - val_loss: 0.5074 - val_fmeasure: 0.9205
>>> F1-score (1): 0.92146 Best (1): - 0.92464 (0.5 : -0.00318)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0174 - fmeasure: 0.9937 - val_loss: 0.5014 - val_fmeasure: 0.9208
>>> F1-score (2): 0.92137 Best (2): - 0.92359 (0.5 : -0.00222)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0165 - fmeasure: 0.9940 - val_loss: 0.5026 - val_fmeasure: 0.9238
>>> F1-score (3): 0.92402 Best (3): - 0.92595 (0.5 : -0.00193)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
59322/59322 [==============================] - 3s - loss: 0.0172 - fmeasure: 0.9937 - val_loss: 0.5476 - val_fmeasure: 0.9122
>>> F1-score (4): 0.91248 Best (4): - 0.91731 (0.5 : -0.00483)

Train on 59322 samples, validate on 3295 samples
Epoch 1/1
5932

In [7]:
if SAVE_MODEL:
    save_simDEF_model(FOLD=FOLD, models=models)
    
if SAVE_EMBEDDINGS:
    save_simDEF_embeddings(FOLD=FOLD, 
                           embedding_layers=embedding_layers,
                           word_indeces=word_indeces, 
                           SUB_ONTOLOGY_work=SUB_ONTOLOGY_work)
    

Saved model0 to disk
Saved model1 to disk
Saved model2 to disk
Saved model3 to disk
Saved model4 to disk
Saved model5 to disk
Saved model6 to disk
Saved model7 to disk
Saved model8 to disk
Saved model9 to disk
The Model and its Weights Are Saved!!
The Word Embeddings Are Saved!!
